# Análisis de datos Experimento 1
Con 10% de hábitat en el paisaje, 10 niveles de intensificación y 100 comunidades.

In [1]:
import numpy as np
import seaborn, math, os
from matplotlib import pyplot as plt
import scipy.stats as stats

In [2]:
%matplotlib inline
plt.style.use('seaborn-whitegrid')
seaborn.set_palette("deep", color_codes=True)

In [3]:
np.set_printoptions(precision=3, suppress=True)

In [4]:
%run medidas_biodiversidad.py

## Funciones para leer datos, etc

In [5]:
def leer_datos(het):
    files = os.listdir("./datos/exp1")
    files.sort()
    files = list(filter(lambda x: x[6] == het, files))
    
    datos = [np.load("./datos/exp1/" + file) for file in files]
    comunidad = [file[9:12] for file in files]
    
    return datos, comunidad

In [6]:
def calcular_medida(medida, datos, paisajes, bio_min = 5.):
    medidas = np.zeros((100, 10))
    for comunidad in range(len(datos)):
        for intensificacion in range(len(datos[0])):
            medidas[comunidad, intensificacion] = medida(
                                                    datos[comunidad][intensificacion], 
                                                    paisajes[intensificacion],
                                                    t=-2, biomasa_min = bio_min)
    return medidas

In [7]:
def calcular_riqueza(datos, paisajes, bio_min = 5.):
    riquezas = np.zeros((100, 10))
    biomasas = np.zeros((100, 10))

    for comunidad in range(len(datos)):
        for intensificacion in range(len(datos[0])):
            biomasas[comunidad, intensificacion], riquezas[comunidad, intensificacion] = riqueza_agricola(
                                                    datos[comunidad][intensificacion], 
                                                    paisajes[intensificacion],
                                                    t=-2, biomasa_min = bio_min)
    return biomasas, riquezas

In [8]:
def calcular_riquezatotal(datos, bio_min = 5.):
    riquezas = np.zeros((100, 10))
    for comunidad in range(len(datos)):
        for intensificacion in range(len(datos[0])):
            riquezas[comunidad, intensificacion] = riqueza_paisaje(
                                                    datos[comunidad][intensificacion],
                                                    t=-2, biomasa_min = bio_min)[1]
    return riquezas

In [9]:
def calcular_promedio(riquezas):
    """Recibe datos de riqueza.
    """
    promedios = np.zeros(10)
    std = np.zeros(10)
    for i in range(10):
        promedios[i] = np.mean(riquezas[riquezas[:, 0]>=0, i])
        std[i] = np.std(riquezas[:, i])
    return promedios, std

## Leer datos, calcular promedios, sd

In [10]:
paisajes = [None] * 5
for het in range(5):
    paisajes[het] = np.load("paisajes_h{}.npy".format(het))

In [ ]:
datos = [None] * 5
riquezas = [None] * 5
biomasas = [None] * 5
riquezas_mean = [None] * 5
riquezas_sd = [None] * 5
biomasas_mean = [None] * 5
biomasas_sd = [None] * 5

prueba = [None] * 5
prueba_mean = [None] * 5
prueba_sd = [None] * 5

for het in range(5):
    datos[het] = leer_datos(str(het))[0]
    
    biomasas[het], riquezas[het] = calcular_riqueza(datos[het], paisajes[het], bio_min = 15)

    riquezas_mean[het], riquezas_sd[het] = calcular_promedio(riquezas[het])

    prueba[het] = calcular_medida(medida_prueba,  datos[het], paisajes[het], bio_min = 15)
    prueba_mean[het], prueba_sd[het] = calcular_promedio(prueba[het])
    
    biomasas_mean[het], biomasas_sd[het] = calcular_promedio(biomasas[het])    

KeyboardInterrupt: 

In [ ]:
riquezatotal = [None] * 5
riquezatotal_mean = [None] * 5
riquezatotal_sd = [None] * 5

for het in range(5):
    riquezatotal[het] = calcular_riquezatotal(datos[het], bio_min = 15)
    riquezatotal_mean[het], riquezatotal_sd[het] = calcular_promedio(riquezatotal[het])  

TypeError: object of type 'NoneType' has no len()

## Graficar

In [ ]:
def graficar(medida, promedios, sd):
    """Graficar promedios de una medida de biodiversidad con curva ajustada ponderada
    """
    y = [None] * 5
    res = [None] * 5
    p = [None] * 5
    r2 = [None] * 5

    x = np.linspace(0, 9, 100)
    xdata = np.array(range(10))
    xlabels = np.array(range(10)) * 10 

    puntos = ["o", "p", "d", "v", "s"]
    colors = ["b", 'g', 'r', 'purple', 'y']

    for het in range(5):
        res[het] = np.polyfit(xdata, promedios[het], 2, w=1/(sd[het])**2)
        p[het] = np.poly1d(res[het])
        r2[het] = 1 - ((np.sum((p[het](range(10)) - promedios[het])**2)) / (np.sum((promedios[het] - np.mean(promedios[het]))**2)))
        
    fig = plt.figure(figsize=(9, 7), dpi = 50)

    for het in range(0, 5):
        y[het] = p[het](x)
        plt.plot(xdata, promedios[het], 'o', color= colors[het]) #data points
        plt.plot(x, y[het], lw = 3, color= colors[het], 
                 label='{} \n $y = {:+.3f} x^2 {:+.3f} x {:+.3f}$ \n $R^2={:.4f}$ '.format(
                 het, float(p[het][2]), float(p[het][1]), float(p[het][0]), float(r2[het]))) #fit

    plt.ylim(-0.05)
    plt.xlim(-0.1, 9.1)
    plt.xticks(xdata, xlabels)
    plt.xlabel("% low quality patches", size = 15)
    plt.ylabel(medida, size = 15)
    plt.title("{} decline in landscapes with different heterogeneity".format(medida), size = 15)

    plt.legend(loc='center left', fontsize = "large", title="heterogeneity", bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

In [ ]:
def graficar_violin(medida, datos, promedios, sd):
    """Graficar datos con violin plot
    """
    y = [None] * 5
    res = [None] * 5
    p = [None] * 5
    r2 = [None] * 5

    x = np.linspace(0, 9, 100)
    xdata = np.array(range(10))
    xlabels = np.array(range(10)) * 10 

    puntos = ["o", "p", "d", "v", "s"]
    colors = ["b", 'g', 'r', 'purple', 'y']

    for het in [4]:
        res[het] = np.polyfit(xdata, promedios[het], 2, w=1/(sd[het])**2)
        p[het] = np.poly1d(res[het])
        r2[het] = 1 - ((np.sum((p[het](range(10)) - promedios[het])**2)) / (np.sum((promedios[het] - np.mean(promedios[het]))**2)))
        
    fig = plt.figure(figsize=(9, 7), dpi = 100)

    for het in [4]:
            y[het] = p[het](x)
            plt.plot(x, y[het], lw = 3, color= colors[het], 
                    label='{} \n $y = {:+.3f} x^2 {:+.3f} x {:+.3f}$ \n $R^2={:.4f}$ '.format(
                    het, float(p[het][2]), float(p[het][1]), float(p[het][0]), float(r2[het]))) #fit
            plt.violinplot(datos[het, :, :], xdata, showmedians=True, showextrema=False, showmeans=True) #data points

    plt.ylim(-0.05)
    plt.xlim(-0.1, 9.1)
    plt.xticks(xdata, xlabels)
    plt.xlabel("% low quality patches", size = 15)
    plt.ylabel(medida, size = 15)
    plt.title("{} decline in landscapes with different heterogeneity".format(medida), size = 15)

    plt.legend(loc='center left', fontsize = "large", title="heterogeneity", bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

## Resultados

¿Cómo se distribuyen los datos? De esto dependen las pruebas de hipótesis que se hacen.

In [ ]:
graficar_violin('Richness', np.array(riquezas), riquezas_mean, riquezas_sd)

### Richness

In [ ]:
graficar('Species richness', riquezas_mean, riquezas_sd)

1. **Species richness declines as a concave down curve as matrix quality decreases, in all heterogeneity levels.**
For each heterogeneity level we fit a second degree polynomial to average species richness using the least-squares method. Following González et al. (2016), we take the quadratic term coefficient as qualitative evidence to characterize the shape of the curves: a negative coefficient indicates that biodiversity decreases as a concave down curve, while a positive coefficient indicates it decreases as a concave up curve. 
The fitted equations are:

Given that the coefficient of the quadratic term of all fitted polynomials is negative, richness declines as a concave down curve in all heterogeneity levels. 

=> Richness is robust in high quality agricultural landscapes

2. **As matrix quality decreases, richness remains higher in high heterogeneity landscapes than in low heterogeneity landscapes.**

    * High heterogeneity landscapes (levels $h_4$, $h_3$ and $h_2$) have similar richness at each degree of intensification, respectively. 
    
    We computed a KW test (non parametric version of Anova) for at each level of intensification to 
    At each level of intensification we tested the null hypothesis that the computed the Kruskal-Wallis test to 
    We use Kruskal test to identify significant differences in biodiversity between heterogeneity levels at each level of intensification. Null hypothesis is that population medians are equal.
    
    
    * There is a significant difference in richness between landscapes at the two extremes of heterogenity (level $h_4$ compared with level $h_0$), along the intensification gradient.
    * The effect of heterogeneity on richness becomes more noticeable as matrix quality decreases. 

3. **Our results were sensitive to the species survival threshold.**
However, richness preserved a concave down curve at increasing orders of magnitude (from 0.0001 to 100), even if there was not a significant difference between heterogeneity levels at some of those values. 

Reading "Using optimization methods...". The authors discuss:
"...in real-world landscapes, a combination of both sharing and sparing strategies is likely needed... Our results can thus be seen as a theoretical example supporting this assertion."

How do we contextualize our results within this argument?? ie basically we are reframing the sharing-sparing debate to account for the compound effect of landscape quality and heterogeneity. Another important property of our model is that we adopt a landscape perspective instead of focusing on a per-patch perspective?? 

Methods

We simulate landscapes with a lattice of 10 by 10 cells with periodic boundary. Each cell represents one patch of either habitat, biodiversity friendly / high quality / low intensity agriculture or biodiversity unfriendly / low quality / high intensity agriculture, and may be occupied by a community, that is, by interacting populations of multiple species. We define the quality of the landscape as the proportion of biodiversity friendly patches. Thus we model the decrease of landscape quality by increasing the proportion of biodiversity unfriendly patches. (//clarify difference between cells / patches).

Discussion
We adapt the model derived by González et al. to study the combined effect of landscape quality and heterogeneity on species richness. 

In [ ]:
graficar('Biomass', biomasas_mean, biomasas_sd)

¿Por qué es importante considerar la distribución espacial de los individuos?

In [ ]:
graficar('Species richness (area)', prueba_mean, prueba_sd)

1. 
**Requiring that species occupy a minimum landscape area as an additional constraint for their survival stressed the effects of heterogeneity on biodiversity.**

    * In low heterogeneity landscapes (levels $h_1$ and $h_0$) richness declines as a concave up curve with decreasing matrix quality; in contrast, richness follows a concave down curve in high heterogeneity landscapes (levels $h_4$, $h_3$ and $h_2$). 

    Under a stricter survival threshold, richness is robust in high heterogeneity landscapes, but it declines drastically with decreasing matrix quality in low heterogeneity landscapes.

    * Again, richness higher with higher het and no difference between higher het levels. 

    * In this case, the effect of heterogeneity on richness is more noticeable at intermediate quality landscapes. 

Falta el experimento control: ¿qué riqueza alcanzan las comunidades cuando no se considera la dinámica espacial, etc?

In [ ]:
graficar('riqueza total', riquezatotal_mean, riquezatotal_sd)

In [ ]:
riquezatotal_mean

### Comentarios

Los puntos más fuertes del modelo son: uso de redes; calidad del manejo, no sólo pérdida de hábitat; ubicarlo en un contexto de conservación. 

Las debilidades: no utilizamos un algoritmo para crear el paisaje, por lo que no podemos controlar parámetros de % hábitat o heterogeneidad/fragmentación; la escala espacial / temporal; 'identidad' de los nodos (M sugiere que para un trabajo posterior se podrían asignar diferentes parámetros de dispersión dependiendo del nivel trófico -- para esto, revisar artículo que compara la supervivencia de diferentes perfiles de especies, que de hecho (creo que es ese) también considera tres tipos de parche (¿cómo miden la calidad de los parches?)).

Preguntas: ¿dependen estas curvas de la cantidad de hábitat? Hacer simulaciones con 70% de hábitat, aunque pensar sobre cómo medir la heterogeneidad. Y revisar ese artículo que discute los niveles de fragmentación donde importa más el manejo. No debemos saturar el texto con información redundante, así que creo que sería conveniente centrarnos en paisajes altamente fragmentados. Los resultados con mayores % hábitat quizá mencionarlo como otra prueba de robustez?? 
También habíamos considerado hacer simulaciones para comparar cómo decae la riqueza conforme se pierde el hábitat, aquí sólo habría dos tipos de parche: hábitat-no hábitat y el objetivo sería comparar nuestro modelo con alguno de los otros, en especial con el que modela la pérdida de hábitat + heterogeneidad en Brasil; replicar resultados de otros es una forma de validación del modelo... pensar al respecto. También: considerar medidas de biodiversidad que consideren la distribución espacial. ¿Cómo argumentar la continuación? 

Para la próxima semana: análisis de sensibilidad de umbral - los patrones que se observan en las curvas (con nivel bio_min = 10 -- parece que es el que usaré) surgen conforme aumenta ese umbral y se mantienen en el rango... por lo menos hasta bio_min = 15. Probar con crecientes órdenes de magnitud. 

¿Qué otros parámetros podrían ser sensibles en el modelo? O que es lo mismo, ¿cuáles son nuestras suposiciones más delicadas que no tienen un sustento biológico/ecológico, que sólo son suposiciones 'numéricas'? Esto es diferente a las debilidades del modelo. Umbral de biomasa min; cantidad de iteraciones entre migración-muerte; las medidas de riqueza se toman como la mediana de la última iteración (se tendría que justificar por qué no el promedio...); la medida de heterogeneidad del paisaje y que fueron creados manualmente; las pruebas estadísticas son básicas y ahora que ya no estamos probando directamente las dos hipótesis no hay una razón para ajustar polinomio cuadrático excepto por continuación del trabajo ya publicado. 

Revisar en el artículo más reciente: redes solo surgen a nivel del paisaje. 

Dónde se ubican los paisajes que se asocian a una u otra estrategia de conservación --> aquí es donde incluiríamos el debate sharing-sparing

¿Cómo concluir? Podríamos sugerir comparar comunidades: ¿una misma comunidad cambia la forma en que decae su riqueza dependiendo del nivel de heterogeneidad del paisaje donde se encuentre? Quizá la mayoría de las comunidades mantienen la forma de la curva, pero con riqueza decreciente; quizá existan comunidades más sensibles que presenten una transición de decaimiento cóncavo hacia arriba -> cóncavo hacia abajo; quizá la mayoría presentan un decaimiento sigmoidal y cambia el umbral de intensificación en el que decaen abruptamente. 

Recordar: por qué interesan patrones generales - difícil encontrar datos para todas las regiones donde también se presenta este fenómeno, tanto de uso de suelo como de las comunidades que los habitan. Falta una referencia acerca de que actividades agrícolas producen distintos patrones de heterogeneidad y sería valioso citar una referencia local. El enfoque teórico nos permite comparar fácilmente diferentes medidas de biodiversidad: es necesario considerar la distribución geográfica.

Fin.

## Identificar diferencias significativas

Note: shape of `np.array(biodiversity_measure)` is `[heterogeneity, communities, intensification]`.

### Do data distribute normally?

We use Shapiro-Wilk test. Null hypothesis is that data subset came from a normally distributed population.

**Result:** Data subsets don't follow a normal distribution. 

In [ ]:
for het in range(5):
    for i in range(10):
        res = 'normal'
        
        data = np.array(riquezas)[het, :, i]
        
        pvalue = stats.shapiro(data)[1]
        if pvalue <= 0.05:
            res = '---NOT NORMAL---'
        print('H{}, i{}, p= {:.4f}, {}'.format(het, i, pvalue, res))

### Do data have the same variance?

We use Levene test, which does not assume normality. Null hypothesis is that population variances are equal. In particular, we want to know whether data subsets from all heterogeneity levels have the same variance at each intensification level.

**Result:** At each intensification level, subsets from all heterogeneity levels have equal variance.

In [ ]:
for i in range(10):
    equal_std = 'equal std'
    pvalue = stats.levene(np.array(riquezas)[0, :, i],
                          np.array(riquezas)[1, :, i],
                          np.array(riquezas)[2, :, i],
                          np.array(riquezas)[3, :, i],
                          np.array(riquezas)[4, :, i], center='median')[1]
    if pvalue <=0.05:
        equal_std = '---std NOT EQUAL---'
    print('i{}, pvalue={:.3f}, {}'.format(i, pvalue, equal_std))

### Differences between heterogeneity levels

We use Kruskal test to identify significant differences in biodiversity between heterogeneity levels at each level of intensification. Null hypothesis is that population medians are equal.

There is no significant difference between heterogeneity levels 4, 3 and 2. 

In [ ]:
# Richness
for i in range(10):
    res = 'no difference'
    pvalue = stats.kruskal(np.array(riquezas)[4, :, i],
                           np.array(riquezas)[3, :, i],
                           np.array(riquezas)[2, :, i])[1]
    if pvalue <=0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue={:.4f}, {}'.format(i, pvalue, res))

In [ ]:
# Area
for i in range(10):
    res = 'no difference'
    pvalue = stats.kruskal(np.array(prueba)[4, :, i],
                           np.array(prueba)[3, :, i],
                           np.array(prueba)[2, :, i])[1]
    if pvalue <=0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue={:.4f}, {}'.format(i, pvalue, res))

Heterogeneity levels 4 (highest) and 0 (lowest) are significantly different at all intensification levels.

In [ ]:
# Richness
for i in range(10):
    res = 'no difference'
    pvalue = stats.kruskal(np.array(riquezas)[4, :, i],
                           np.array(riquezas)[0, :, i])[1]
    if pvalue <=0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue={:.4f}, {}'.format(i, pvalue, res))

In [ ]:
# Area
for i in range(10):
    res = 'no difference'
    pvalue = stats.kruskal(np.array(prueba)[4, :, i],
                           np.array(prueba)[0, :, i])[1]
    if pvalue <=0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue={:.4f}, {}'.format(i, pvalue, res))

In [ ]:
# Richness
for i in range(10):
    res = 'no difference'
    pvalue = stats.f_oneway(np.array(riquezas)[4, :, i],
                            np.array(riquezas)[0, :, i])[1]
    if pvalue <=0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue={:.4f}, {}'.format(i, pvalue, res))

At what intensification levels are heterogeneity levels 4, 1 and 0 different from each other? ie At what intensification levels is het1 different from het4 and from het0?

In [ ]:
# Richness
for i in range(10):
    res = '-'
    pvalue_14 = stats.kruskal(np.array(riquezas)[4, :, i],
                               np.array(riquezas)[1, :, i])[1]
    
    pvalue_10 = stats.kruskal(np.array(riquezas)[0, :, i],
                               np.array(riquezas)[1, :, i])[1]
    if pvalue_14 <=0.05 and pvalue_10 <= 0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue_14={:.4f}, pvalue_10={:.4f}, {}'.format(i, pvalue_14, pvalue_10, res))

In [ ]:
# Area
for i in range(10):
    res = '-'
    pvalue_14 = stats.kruskal(np.array(prueba)[4, :, i],
                               np.array(prueba)[1, :, i])[1]
    
    pvalue_10 = stats.kruskal(np.array(prueba)[0, :, i],
                               np.array(prueba)[1, :, i])[1]
    if pvalue_14 <=0.05 and pvalue_10 <= 0.05:
        res = '---DIFFERENT---'
    print('i{}, pvalue_14={:.4f}, pvalue_10={:.4f}, {}'.format(i, pvalue_14, pvalue_10, res))

In [ ]:
# Quiero saber si para un mismo nivel de het, los datos de todos los niveles de intensificación tienen la misma std
for het in range(5):
    equal_std = 'equal std'
    pvalue = stats.levene(np.array(riquezas)[het, :, 0],
                      np.array(riquezas)[het, :, 1],
                      np.array(riquezas)[het, :, 2],
                      np.array(riquezas)[het, :, 3],
                      np.array(riquezas)[het, :, 4],
                      np.array(riquezas)[het, :, 5],
                      np.array(riquezas)[het, :, 6],
                      np.array(riquezas)[het, :, 7],
                      np.array(riquezas)[het, :, 8],
                      np.array(riquezas)[het, :, 9], center='median')[1]
    if pvalue <=0.05:
        equal_std = '---std NOT EQUAL---'
    print('h{}, pvalue={:.3f}, {}'.format(het, pvalue, equal_std))

In [ ]:
?stats.kruskal